# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests


# Import API key
from api_keys import geoapify_key

In [3]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,tiksi,71.6872,128.8694,250.10,100,100,3.01,RU,1702353204
1,1,nuuk,64.1835,-51.7216,274.49,93,100,20.06,GL,1702353205
2,2,hamilton,39.1834,-84.5333,273.70,83,0,3.09,US,1702352493
3,3,waitangi,-43.9535,-176.5597,288.05,93,100,3.13,NZ,1702353206
4,4,ankang,32.6800,109.0172,283.24,68,98,1.21,CN,1702353206


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [4]:
# Configure the map plot
map_plot = city_data_df.hvplot.scatter(
    x='Lng',
    y='Lat',
    c='Max Temp',
    cmap='viridis',
    colorbar=True,
    size='Humidity',
    alpha=0.7,
    hover_cols=['City', 'Country', 'Max Temp', 'Humidity']
)

# Customize the plot (optional)
map_plot = map_plot.opts(
    title='City Temperature and Humidity Map',
    width=800,
    height=500
)

# Display the map
map_plot

:Scatter   [Lng]   (Lat,Max Temp,Humidity,City,Country)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [5]:
# Filter cities with no cloudiness
cloudiness_zero_df = city_data_df[city_data_df['Cloudiness'] == 0]

# Drop any rows with null values (if needed)
cloudiness_zero_df = cloudiness_zero_df.dropna()

# Display the filtered DataFrame
cloudiness_zero_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
2,2,hamilton,39.1834,-84.5333,273.70,83,0,3.09,US,1702352493
10,10,kodiak,57.7900,-152.4072,269.88,63,0,2.57,US,1702353209
13,13,plymouth,41.8334,-70.8328,277.10,70,0,6.17,US,1702353211
25,25,de queen,34.0379,-94.3413,280.94,87,0,0.00,US,1702353216
32,32,ovalle,-30.5983,-71.2003,292.49,63,0,1.05,CL,1702353220


### Step 3: Create a new DataFrame called `hotel_df`.

In [6]:
# Use the Pandas copy function to create a DataFrame called hotel_df to store the city, country, coordinates, and humidity
# Change from using cloudiness_zero_df to city_data_df
hotel_df = city_data_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame to store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ""

# Display sample data
hotel_df.head()


,City,Country,Lat,Lng,Humidity,Hotel Name
0,tiksi,RU,71.6872,128.8694,100,
1,nuuk,GL,64.1835,-51.7216,93,
2,hamilton,US,39.1834,-84.5333,83,
3,waitangi,NZ,-43.9535,-176.5597,93,
4,ankang,CN,32.6800,109.0172,68,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [9]:
import requests

# Set parameters to search for a hotel
params = {
    "type": "accommodation.hotel",
    "limit": 10,  # Adjust if you need more results
    "apiKey": geoapify_key  # Your Geoapify API key
}

print("Starting hotel search")

successful_responses = 0  # Initialize counter for successful responses

for index, row in hotel_df.iterrows():
    lat = row['Lat']
    lng = row['Lng']

    # Use 'filter' with 'circle' to specify the radius
    params["filter"] = f"circle:{lng},{lat},10000"  # 10,000 meters radius

    base_url = "https://api.geoapify.com/v2/places"

    response = requests.get(base_url, params=params)

    print(f"Status Code: {response.status_code}")
    print(f"Response Text: {response.text}")

    if response.status_code == 200:
        name_address = response.json()

        try:
            hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
        except (KeyError, IndexError):
            hotel_df.loc[index, "Hotel Name"] = "No hotel found within 10km"
        
        print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")
        
        successful_responses += 1  # Increment the counter

        # Check if 50 successful responses have been reached
        if successful_responses >= 50:
            print("Reached 50 successful hotel searches. Stopping the search.")
            break
    else:
        hotel_df.loc[index, "Hotel Name"] = "Error fetching hotel data"
        print(f"Error fetching data for {hotel_df.loc[index, 'City']}")

print(hotel_df)


Starting hotel search
Status Code: 200
Response Text: {"type":"FeatureCollection","features":[{"type":"Feature","properties":{"name":"Арктика","country":"Russia","country_code":"ru","region":"Far Eastern Federal District","state":"Sakha Republic","county":"Bulunsky Ulus","city":"Tiksi","suburb":"Tiksi-3","street":"Polyarnoi aviatsii street","lon":128.87147940836854,"lat":71.6922024,"formatted":"Арктика, Polyarnoi aviatsii street, Tiksi-3, Tiksi, Sakha Republic, Russia","address_line1":"Арктика","address_line2":"Polyarnoi aviatsii street, Tiksi-3, Tiksi, Sakha Republic, Russia","categories":["accommodation","accommodation.hotel","building","building.accommodation"],"details":["details.accommodation","details.building"],"datasource":{"sourcename":"openstreetmap","attribution":"© OpenStreetMap contributors","license":"Open Database Licence","url":"https://www.openstreetmap.org/copyright","raw":{"name":"Арктика","stars":2,"osm_id":141488442,"tourism":"hotel","building":"yes","osm_type":"w"

### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [10]:

# Merge hotel_df with city_data_df
merged_df = city_data_df.merge(hotel_df[['City', 'Hotel Name']], on='City', how='left')
import hvplot.pandas  # Ensure hvplot is imported


# Configure the map plot using 'merged_df'
map_plot = merged_df.hvplot.scatter(
    x='Lng',
    y='Lat',
    c='Max Temp',  # Color the points based on Max Temp
    cmap='viridis',
    colorbar=True,
    size='Humidity',  # Size of the points based on Humidity
    alpha=0.7,
    hover_cols=['City', 'Country', 'Max Temp', 'Humidity', 'Hotel Name']  # Include 'Hotel Name' in hover info
)

# Customize the plot
map_plot = map_plot.opts(
    title='City Temperature, Humidity, and Nearest Hotel Map',
    width=800,
    height=500,
    tools=['hover']  # Ensure hover tool is activated
)

# Display the map
map_plot


:Scatter   [Lng]   (Lat,Max Temp,Humidity,City,Country,Hotel Name)